In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/pokemon-types/Pokemon Type Combined Scores.csv
/kaggle/input/pokemon-types/Pokemon Type Score.csv
/kaggle/input/pokemon-types/Pokemon Type Stats.csv
/kaggle/input/pokemon-types/Pokemon Type Chart.csv
/kaggle/input/pokemon/Pokemon.csv


In [3]:
!pip install -Iv pulp==1.6.8 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 26.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 7.9 MB/s eta 0:00:00
  Created wheel for pulp: filename=PuLP-1.6.8-py3-none-any.whl size=12269582 sha256=741903b5da604768010a8ae120e77c256fbd6a477119e8a8dd2fcd6456b6a633
  Stored in directory: /root/.cache/pip/wheels/ef/80/90/298f850e9f5e1f1a41725c8b632c8652f4085a81f2374683db
Successfully built pulp
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
beatrix-jupyterlab 3.1.7 requires google-cloud-bigquery-storage, which is not installed.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 1.12.11 which is incompatible.
apache-beam 2.40.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.5.1 which is incompatible.
apache-beam 

In [4]:
import numpy as np
import pandas as pd

poke_df = pd.read_csv("../input/pokemon/Pokemon.csv")

In [5]:
poke_df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [6]:
poke_df = poke_df.rename(columns={"#": "No"})
poke_df.head()

,No,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [7]:
unova = list(range(495,650))

genV_dex = unova

unova_df = poke_df[poke_df.No.isin(genV_dex)]
unova_df

,No,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
554,495,Snivy,Grass,NaN,308,45,45,55,45,55,63,5,False
555,496,Servine,Grass,NaN,413,60,60,75,60,75,83,5,False
556,497,Serperior,Grass,NaN,528,75,75,95,75,95,113,5,False
557,498,Tepig,Fire,NaN,308,65,63,45,45,45,45,5,False
558,499,Pignite,Fire,Fighting,418,90,93,55,70,55,55,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,647,KeldeoOrdinary Forme,Water,Fighting,580,91,72,90,129,90,108,5,False
714,647,KeldeoResolute Forme,Water,Fighting,580,91,72,90,129,90,108,5,False
715,648,MeloettaAria Forme,Normal,Psychic,600,100,77,77,128,128,90,5,False
716,648,MeloettaPirouette Forme,Normal,Fighting,600,100,128,90,77,77,128,5,False


In [8]:
unova_dex = unova_df.drop_duplicates(subset=['No'])
unova_dex = unova_dex.rename(columns={"Type 1": "Type1"})
unova_dex = unova_dex.rename(columns={"Type 2": "Type2"})
unova_dex

,No,Name,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
554,495,Snivy,Grass,NaN,308,45,45,55,45,55,63,5,False
555,496,Servine,Grass,NaN,413,60,60,75,60,75,83,5,False
556,497,Serperior,Grass,NaN,528,75,75,95,75,95,113,5,False
557,498,Tepig,Fire,NaN,308,65,63,45,45,45,45,5,False
558,499,Pignite,Fire,Fighting,418,90,93,55,70,55,55,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,645,LandorusIncarnate Forme,Ground,Flying,600,89,125,90,115,80,101,5,True
710,646,Kyurem,Dragon,Ice,660,125,130,90,130,90,95,5,True
713,647,KeldeoOrdinary Forme,Water,Fighting,580,91,72,90,129,90,108,5,False
715,648,MeloettaAria Forme,Normal,Psychic,600,100,77,77,128,128,90,5,False


In [9]:
unova_dex = unova_dex[unova_dex.Legendary == False]
unova_dex

,No,Name,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
554,495,Snivy,Grass,NaN,308,45,45,55,45,55,63,5,False
555,496,Servine,Grass,NaN,413,60,60,75,60,75,83,5,False
556,497,Serperior,Grass,NaN,528,75,75,95,75,95,113,5,False
557,498,Tepig,Fire,NaN,308,65,63,45,45,45,45,5,False
558,499,Pignite,Fire,Fighting,418,90,93,55,70,55,55,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,636,Larvesta,Bug,Fire,360,55,85,55,50,55,60,5,False
698,637,Volcarona,Bug,Fire,550,85,60,65,135,105,100,5,False
713,647,KeldeoOrdinary Forme,Water,Fighting,580,91,72,90,129,90,108,5,False
715,648,MeloettaAria Forme,Normal,Psychic,600,100,77,77,128,128,90,5,False


In [10]:
typeChart = pd.read_csv("../input/pokemon-types/Pokemon Type Chart.csv", index_col = [0])
typeChart

,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy
Normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5,1.0
Fire,1.0,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0,1.0
Water,1.0,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0,1.0
Electric,1.0,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0
Grass,1.0,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5,1.0
Ice,1.0,0.5,0.5,1.0,2.0,0.5,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0
Fighting,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,0.5,0.5,0.5,2.0,0.0,1.0,2.0,2.0,0.5
Poison,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.5,0.5,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.0,2.0
Ground,1.0,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,0.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0,1.0
Flying,1.0,1.0,1.0,0.5,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,0.5,1.0


In [11]:
types = [ 'Normal', 'Bug', 'Electric', 'Ground', 'Flying',
         'Ice', 'Dragon', 'Fighting', 'Dark', 'Ghost', 'Psychic']
gymEliteFour = typeChart[[c for c in typeChart.columns if c in types]]
gymEliteFour

,Normal,Electric,Ice,Fighting,Ground,Flying,Psychic,Bug,Ghost,Dragon,Dark
Normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
Fire,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0
Water,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0
Electric,1.0,0.5,1.0,1.0,0.0,2.0,1.0,1.0,1.0,0.5,1.0
Grass,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,0.5,1.0
Ice,1.0,1.0,0.5,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0
Fighting,2.0,1.0,2.0,1.0,1.0,0.5,0.5,0.5,0.0,1.0,2.0
Poison,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,0.5,1.0,1.0
Ground,1.0,2.0,1.0,1.0,1.0,0.0,1.0,0.5,1.0,1.0,1.0
Flying,1.0,0.5,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0


In [12]:
strengths = []
numbers = []
names = []
totalStats = []
i = 0
for row in unova_dex.itertuples(index=True, name='Pandas'):
    numbers.append(row.No)
    names.append(row.Name)
    
    strengths.append([1 if s >= 2.0 else 0 for s in list(gymEliteFour.loc[row.Type1])])
    totalStats.append(row.Total)
    
    try:
        strengths[i] = list(np.add(strengths[i], [1 if s >= 2.0 else 0 for s in list(gymEliteFour.loc[row.Type2])]))
        
    except:
        KeyError
        
    i += 1
    
advantage = [[1 if r >= 1 else 0 for r in strengths[i]] for i in range(146)]

starterNames = ['Snivy', 'Servine', 'Serperior', 
                'Tepig', 'Pignite', 'Emboar',
               'Oshawott', 'Dewott', 'Samurott']

starterIndices = []
for pkmn in starterNames:
    starterIndices.append(names.index(pkmn))

from pulp import *
mythicalNames = ['MeloettaAria Forme', 'Genesect']

mythicalIndices = []
for pkmn in mythicalNames:
    mythicalIndices.append(names.index(pkmn))


best_team = LpProblem("Optimal_Pokemon_Team", LpMaximize)

P = LpVariable.dicts("P", range(146), cat=LpBinary) # we have 206 Pokemon to consider

best_team += lpSum([totalStats[pkmn] * P[pkmn] for pkmn in range(146)])

best_team += lpSum([P[pkmn] for pkmn in range(146)]) == 6

best_team += lpSum([P[pkmn] for pkmn in starterIndices]) == 1

for i in range(11):
    best_team += lpSum([advantage[pkmn][i] * P[pkmn] for pkmn in range(146)]) >= 1

best_team.solve()
for i in range(146):
    if P[i].value() == 1:
        print(names[i])


Emboar
Seismitoad
Archeops
Hydreigon
MeloettaAria Forme
Genesect


In [13]:
from pulp import *

mythicalNames = ['MeloettaAria Forme', 'Genesect', 'KeldeoOrdinary Forme', 'Hydreigon', 'Volcarona','Archeops', 'Braviary'
]

mythicalIndices = []
for pkmn in mythicalNames:
    mythicalIndices.append(names.index(pkmn))

best_team = LpProblem("Optimal_Pokemon_Team", LpMaximize)

P = LpVariable.dicts("P", range(146), cat=LpBinary)

# Define the objective function to maximize
best_team += lpSum([totalStats[pkmn] * P[pkmn] for pkmn in range(146)])

best_team += lpSum([P[pkmn] for pkmn in range(146)]) == 6

best_team += lpSum([P[pkmn] for pkmn in starterIndices]) == 1

best_team += lpSum([P[pkmn] for pkmn in mythicalIndices]) == 0

for i in range(11):
    best_team += lpSum([advantage[pkmn][i] * P[pkmn] for pkmn in range(146)]) >= 1

best_team.solve()

for i in range(146):
    if P[i].value() == 1:
        print(names[i])

Emboar
Krookodile
Vanilluxe
Chandelure
Haxorus
Mandibuzz
